In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sys
from scipy.constants import k as k_B
from scipy.constants import N_A
import seaborn as sns
import re
from scipy.optimize import curve_fit
import matplotlib.patches as mpatches

In [2]:
BOX_LENGTH = 0
def read_xyz(file):
    global BOX_LENGTH
    V = []
    R = []
    R_red = []
    opt = []
    n = 0
    frame = {'x':np.array([]), 'y':np.array([])}

    with open(file) as f:
        coords = []
        for line in f:
            line = line.strip()
            if re.match(r'^\d+$', line):
                if len(V) > 0:
                    df = pd.DataFrame(coords, columns=['x','y','z'])

                    xdata = df[['x','y']].values
                    ydata = df['z'].values
                    popt, pcov = curve_fit(lambda x, a, b, c: a*x[:,0] + b*x[:,1] + c, xdata, ydata)
                    opt.append(popt)

                    origin = df.mean()
                    
                    r = (df['x'] - origin['x'])**2 + (df['y'] - origin['y'])**2
                    percentile = np.percentile(r, 75)
                    R_red.append(np.sqrt(percentile))
                    R.append(np.sqrt(r.mean()))

                    coords = []
                    if n % 100 == 0:
                        frame['x'] = np.append(frame['x'], df['x'].values - origin['x'])
                        frame['y'] = np.append(frame['y'], df['y'].values - origin['y'])
                    n += 1

            elif line.startswith('X'):
                coords.append([float(x) for x in re.split(r'\s+', line)[1:]])
            else:
                x,y,z = [float(x) for x in re.split(r'\s+', line)]
                V.append(x*y*z)
                BOX_LENGTH = (BOX_LENGTH*n + z)/ (n+1)
    opt=np.array(opt)
    frame = pd.DataFrame(frame)

    print(f'V = {np.mean(V):.2f} nm^3 +- {np.std(V):.2f} nm^3')
    return np.array(R), np.array(R_red), opt, frame

In [46]:
filenm = 'COLVARS/test50_COLVAR'
with open(filenm) as f:
    #Read the first line
    line = f.readline()
    header=line.split()[2:]

CG = pd.read_csv(filenm, sep=' ', comment='#', names=header).set_index('time')
CG.head()

,dist_A,dist_I1,dist_B
time,,,
0.0,0.065395,0.250466,0.277611


In [ ]:
CG.plot(y=['dist_A','dist_B','dist_I1'])

In [48]:
R_tail, R_tail_red, opt, frame = read_xyz('COLVARS/test50_tail.xyz')
R_core, R_core_red, opt, frame = read_xyz('COLVARS/test50_core.xyz')
RC = pd.DataFrame({'R':R_core, 'time':np.array(range(R_core.shape[0]))*0.002*10})
RT = pd.DataFrame({'R':R_tail, 'time':np.array(range(R_tail.shape[0]))*0.002*10})


V = 6116.09 nm^3 +- 6.85 nm^3
V = 6116.09 nm^3 +- 6.85 nm^3


In [ ]:
RC['diff'] = RC['R'] - RC['R'][0]
RT['diff'] = RT['R'] - RT['R'][0]
sns.lineplot(RC, x='time', y='diff')
sns.lineplot(RT, x='time', y='diff')
plt.ylabel('Radius (nm)')
plt.xlabel('Time (ns)')